In [1]:
!pip install -U accelerate bitsandbytes peft transformers datasets trl git-lfs wandb flash-attn sql-metadata

  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
  Using cached bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl (102.2 MB)
  Using cached peft-0.10.0-py3-none-any.whl (199 kB)
  Using cached transformers-4.39.2-py3-none-any.whl (8.8 MB)
  Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
  Using cached trl-0.8.1-py3-none-any.whl (225 kB)
  Using cached git_lfs-1.6-py2.py3-none-any.whl (5.6 kB)
  Using cached wandb-0.16.5-py3-none-any.whl (2.2 MB)
  Using cached flash_attn-2.5.6.tar.gz (2.5 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-kdbsaw6j/flash-attn_6131aebaa0784e5bab25e392

In [2]:
import torch
import re
import pandas as pd
from transformers import AutoTokenizer, BitsAndBytesConfig
from transformers import AutoModelForCausalLM
from peft import PeftModel
from torch import cuda
from tqdm import tqdm

In [4]:
torch.cuda.is_available()

True

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "deepseek-ai/deepseek-coder-6.7b-instruct"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    attn_implementation="flash_attention_2", # use with amper architecture
    torch_dtype=torch.bfloat16,
    #quantization_config=bnb_config, # use when low on memory
    device_map = "auto"
)

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like deepseek-ai/deepseek-coder-6.7b-instruct is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
model = PeftModel.from_pretrained(model, "/content/final_checkpoint",torch_dtype = torch.bfloat16)
model = model.merge_and_unload()
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
tokenizer.encode(' ;')

In [ ]:
from transformers import StoppingCriteria
class EosListStoppingCriteria(StoppingCriteria):
    def __init__(self, eos_sequence = [6203]):
        self.eos_sequence = eos_sequence

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids
    
def append_string_to_file(text, file_path):
  with open(file_path, 'a') as file:
      file.write(text + '\n')

def remove_spaces(text):
  return re.sub(r'\s+', ' ', text)

def call_mistral(inputs):
  output_tokens = model.generate(inputs, max_new_tokens=250, do_sample=False, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id, stopping_criteria = [EosListStoppingCriteria()])
  return tokenizer.decode(output_tokens[0][len(inputs[0]):], skip_special_tokens=True)

In [5]:
df = pd.read_csv("validation_dataset.csv")
results = []
for index, row in tqdm(df.iterrows(), total=len(df)):
  question = row['question']
  query = row['query']
  database_schema = row['database_schema']
  db_id = row['db_id']
  user_message = f"""Given the following SQL tables, your job is to generate the Sqlite SQL query given the user's question.
Put your answer inside the ```sql and ``` tags.
{database_schema}
###
Question: {question}
"""
  messages = [
      {"role": "user", "content": user_message.strip()}
  ]
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt",add_generation_prompt=True,tokenize = True).to(model.device)
  response = call_mistral(inputs)
  if ";" in response:
    response = response.split(";")[0]
    if "```sql" in response:
      response = response.split("```sql")[1]
  response = re.sub(r'\s+', ' ', response).strip()
  print("\n")
  print(response)
  print(query)
  print("============================")
  results.append([response, query,row['question'],row['db_id']])
  new_df = pd.DataFrame(results, columns = ['generated_query','reference_query','question','db_id'])

FileNotFoundError: [Errno 2] No such file or directory: 'validation_dataset.csv'

In [ ]:
for index, row in new_df.iterrows():
  print(f"Processing the {index}th rows")
  if pd.isna(row['generated_query']):
    print(row['generated_query'])
    sql_query = input("give me the correct SQL query")
    sql_query = remove_spaces(sql_query)
    append_string_to_file(sql_query, "Predicted.txt")
    append_string_to_file(row['reference_query'] + "\t" + row['db_id'], "Gold.txt")
  elif row['generated_query'][:6] == "SELECT":
    append_string_to_file(remove_spaces(row['generated_query']), "Predicted.txt")
    append_string_to_file(row['reference_query'] + "\t" + row['db_id'], "Gold.txt")
  else:
    print(row['generated_query'])
    sql_query = input("give me the correct SQL query")
    sql_query = remove_spaces(sql_query)
    append_string_to_file(sql_query, "Predicted.txt")
    append_string_to_file(row['reference_query'] + "\t" + row['db_id'], "Gold.txt")